<a href="https://colab.research.google.com/github/sakshik2004/FMML/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np



In [2]:
# Load the dataset
file_path = '/content/seattle-weather (1).csv'
data = pd.read_csv(file_path)



In [3]:
# Display the first few rows of the dataset
print(data.head())



         date  precipitation  temp_max  temp_min  wind  weather
0  2012-01-01            0.0      12.8       5.0   4.7  drizzle
1  2012-01-02           10.9      10.6       2.8   4.5     rain
2  2012-01-03            0.8      11.7       7.2   2.3     rain
3  2012-01-04           20.3      12.2       5.6   4.7     rain
4  2012-01-05            1.3       8.9       2.8   6.1     rain


In [4]:
# Assuming the dataset has columns ['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']
# Convert date to datetime
data['date'] = pd.to_datetime(data['date'])



In [5]:
# Encode the weather column (assuming it's a categorical variable)
data['weather'] = data['weather'].astype('category').cat.codes



In [6]:
# Feature selection
features = data[['precipitation', 'temp_max', 'temp_min', 'wind']]
labels = data['weather']



In [7]:
# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)



In [8]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [9]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)




In [10]:
import torch.nn as nn
import torch.optim as optim



In [11]:
class WeatherRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(WeatherRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

input_size = X_train.shape[1]
hidden_size = 50
output_size = len(data['weather'].unique())
num_layers = 2

model = WeatherRNN(input_size, hidden_size, output_size, num_layers)



In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [13]:
# Training loop
num_epochs = 100
batch_size = 64

for epoch in range(num_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        # Forward pass
        outputs = model(batch_x.unsqueeze(1))
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



model.eval()
with torch.no_grad():
    test_outputs = model(X_test.unsqueeze(1))
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy*100:.2f}%')


Epoch [10/100], Loss: 0.6821
Epoch [20/100], Loss: 0.4544
Epoch [30/100], Loss: 0.5283
Epoch [40/100], Loss: 0.7721
Epoch [50/100], Loss: 0.3036
Epoch [60/100], Loss: 0.9246
Epoch [70/100], Loss: 0.7042
Epoch [80/100], Loss: 0.3013
Epoch [90/100], Loss: 0.2343
Epoch [100/100], Loss: 0.3514
Accuracy: 82.59%
